# Research on D-Wave's SDK Ocean

In [5]:
#firstly, installing Ocean
#pip install dwave-ocean-sdk

In [1]:
#importing all required libraries
import networkx as nx
import matplotlib
matplotlib.use("agg")
from matplotlib import pyplot as plt

from collections import defaultdict

from dimod.reference.samplers import ExactSolver
import dwave_networkx as dnx
from dwave.system import DWaveSampler, EmbeddingComposite

In [2]:
s5 = nx.star_graph(4)
sampler = ExactSolver()
print(dnx.min_vertex_cover(s5, sampler))
#sampler = EmbeddingComposite(DWaveSampler(token=token))
sampler = EmbeddingComposite(DWaveSampler())
print(dnx.min_vertex_cover(s5, sampler))

[0]
[0]


# Max cut problem

In [3]:
Q = defaultdict(int)
print(Q)

defaultdict(<class 'int'>, {})


In [4]:
# ------- Set up our graph -------

# Create empty graph
G = nx.Graph()

# Add edges to the graph
G.add_edges_from([(1,2),(1,3),(2,4),(3,4),(3,5),(4,5)])
# G.add_edges_from([(1,3),(1,6),(1,9),(2,7),(2,4),(3,8),(3,11),(4,6),(4,9),(4,10),(5,6),(6,12),(6,13),(7,11),(8,10),(9,11),(12,13)])
# G.add_edges_from([(1,7),(1,9),(2,3),(2,13),(3,12),(4,11),(5,11),(6,13),(8,12),(10,11),(10,14)])

# ------- Set up our QUBO dictionary -------

# Initialize the Q matrix
Q = defaultdict(int)

# Update Q matrix for every edge in the graph
for i, j in G.edges:
    Q[(i,i)]+= -1
    Q[(j,j)]+= -1
    Q[(i,j)]+= 2
print(Q)

defaultdict(<class 'int'>, {(1, 1): -2, (2, 2): -2, (1, 2): 2, (3, 3): -3, (1, 3): 2, (4, 4): -3, (2, 4): 2, (3, 4): 2, (5, 5): -2, (3, 5): 2, (4, 5): 2})


In [5]:
# ------- Run our QUBO on the QPU -------
# Set up QPU parameters
chainstrength = 8
numruns = 100

# Run the QUBO on the solver from your config file
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chainstrength,
                               num_reads=numruns,
                               label='Example - Maximum Cut')

# ------- Print results to user -------
print('-' * 60)
print('{:>15s}{:>15s}{:^15s}{:^15s}'.format('Set 0','Set 1','Energy','Cut Size'))
print('-' * 60)
for sample, E in response.data(fields=['sample','energy']):
    S0 = [k for k,v in sample.items() if v == 0]
    S1 = [k for k,v in sample.items() if v == 1]
    print('{:>15s}{:>15s}{:^15s}{:^15s}'.format(str(S0),str(S1),str(E),str(int(-1*E))))

# ------- Display results to user -------
# Grab best result
# Note: "best" result is the result with the lowest energy
# Note2: the look up table (lut) is a dictionary, where the key is the node index
#   and the value is the set label. For example, lut[5] = 1, indicates that
#   node 5 is in set 1 (S1).
lut = response.first.sample

# Interpret best result in terms of nodes and edges
S0 = [node for node in G.nodes if not lut[node]]
S1 = [node for node in G.nodes if lut[node]]
cut_edges = [(u, v) for u, v in G.edges if lut[u]!=lut[v]]
uncut_edges = [(u, v) for u, v in G.edges if lut[u]==lut[v]]

# Display best result
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, nodelist=S0, node_color='r')
nx.draw_networkx_nodes(G, pos, nodelist=S1, node_color='c')
nx.draw_networkx_edges(G, pos, edgelist=cut_edges, style='dashdot', alpha=0.5, width=3)
nx.draw_networkx_edges(G, pos, edgelist=uncut_edges, style='solid', width=3)
nx.draw_networkx_labels(G, pos)

filename = "maxcut_plot.png"
plt.savefig(filename, bbox_inches='tight')
print("\nYour plot is saved to {}".format(filename))

------------------------------------------------------------
          Set 0          Set 1    Energy        Cut Size    
------------------------------------------------------------
      [2, 3, 5]         [1, 4]     -5.0             5       
      [1, 4, 5]         [2, 3]     -5.0             5       
         [1, 4]      [2, 3, 5]     -5.0             5       
         [2, 3]      [1, 4, 5]     -5.0             5       

Your plot is saved to maxcut_plot.png


## Constrained environment. Knapsack problem

In [6]:
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel
from dwave.system import LeapHybridCQMSampler

In [7]:
values = [2,3,4]
weights = [5,3,1]
max_weight = 5

In [8]:
print(f"solving for {len(values)} items")
print("build the model")
cqm = ConstrainedQuadraticModel()
obj = BinaryQuadraticModel(vartype="BINARY")
constraint = QuadraticModel()

for i in range(len(values)):
    obj.add_variable(i)
    obj.set_linear(i, -values[i])
    constraint.add_variable("BINARY", i)
    constraint.set_linear(i, weights[i])

cqm.set_objective(obj)
cqm.add_constraint(constraint, sense="<=", rhs=max_weight, label="capacity")
print(cqm)

sampler = LeapHybridCQMSampler()
print(f"submit model to solver {sampler.solver.name}")
sampleset = sampler.sample_cqm(cqm, label="knapsack problem")

print("parse the solver output")
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
if not len(feasible_sampleset):
    raise ValueError("No feasible solution found")
best = feasible_sampleset.first

selected_items = [key for key, val in best.sample.items() if val == 1.0]
print(selected_items)
for key, val in best.sample.items():
    print(key, val)

solving for 3 items
build the model
Constrained quadratic model: 3 variables, 1 constraints, 6 biases

Objective
  -2*Binary(0) - 3*Binary(1) - 4*Binary(2)

Constraints
  capacity: 5*Binary(0) + 3*Binary(1) + Binary(2) <= 5.0

Bounds

submit model to solver hybrid_constrained_quadratic_model_version1
parse the solver output
[1, 2]
0 0.0
1 1.0
2 1.0
